Por alguna razón, si leemos (read) dos veces el archivo (arx) abierto, la segunda vez nos da un array vacio. 
Asi que mas bien lo leemos y lo cerramos de una.
Al parecer es porque read avanza en el registor, así que si no especificamos el número de bytes o pasos, pus nos leemos todo y quedamos al final. De todas formas por seguridad de cosas que no entiendo, dejemos el close.

In [2]:
arx=open("../ABFiles/Eduardo/CA3_50UMcarbachol_EMAD_060319_raton.abf", "r")
databin=read(arx)
close(arx)

In [3]:
typeof(databin) # Los datos estan en bytes separados en palabras de 8 bytes.

Array{UInt8,1}

In [4]:
head=databin[1:4]

4-element Array{UInt8,1}:
 0x41
 0x42
 0x46
 0x32

In [5]:
string(head[4]), string(50, base=7), Char(head[4])

("50", "101", '2')

Lo que descubrió swharden es que el 4 número nos dice si es un abf1 o un abf2. Si el número es un espacio (en ASCII es el número 32) es un abf1. Si es un 50 (ASCII para "2") es un abf2:
 - ascii 32 es espacio -> abf1
 - ascii 50 es 2 -> abf2

In [6]:
convert(Int,databin[4]) #aquí vemos que tipo de archivo es. Curiosamente 0x32 es 50, no 32. ( 3 * 16 +2 )

50

In [7]:
map(Char, head) # and here it is

4-element Array{Char,1}:
 'A'
 'B'
 'F'
 '2'

In [8]:
if( Char(head[4])=='2')
    println( "Tenemos un archivo abf2")
elseif ( Char(head[4])==' ')
    println( "Tenemos un archivo abf1")
else
    println( "no tengo zorra idea de que es esto")
end

Tenemos un archivo abf2


In [10]:
aver=databin[13:16]

4-element Array{UInt8,1}:
 0x01
 0x00
 0x00
 0x00

In [11]:
averviendo=IOBuffer(aver)
read(averviendo, Int32)

1

In [52]:
averviendo=IOBuffer(head)
encabezado=read(averviendo, String)

"ABF "

In [53]:
typeof(encabezado)

String

In [14]:
function readfrombyn(data::Array{UInt8,1}, startbyte::Int, endbyte::Int, typeout::Type)
    # reads byte data for an array, and interprets as typeout, returnint it.
    rango=startbyte:endbyte
    aux=IOBuffer(data[rango])
    result=read(aux, typeout)
    return result
end

readfrombyn (generic function with 1 method)

In [15]:
readfrombyn(databin, 1, 4, String)

"ABF2"

In [39]:
aux=IOBuffer(databin[5:8])
read(aux, Float16)

Float16(0.0)

In [17]:
fFileVersionNumber = readfrombyn(databin, 5,5, Float16)

EOFError: EOFError: read end of file

# Encabezado de un ABF1

mmmm tal vez podemos hacer un programa QUE AUTOMATICE esta conversion!

In [ ]:
fFileSignature = readfrombyn(f, 1, 4, String )
fFileVersionNumber = readfrombyn(f, 5,8, Float16)
nOperationMode = readfrombyn(f, "h", 8)
lActualAcqLength = readfrombyn(f, "i", 10)
nNumPointsIgnored = readfrombyn(f, "h", 14)
lActualEpisodes = readfrombyn(f, "i", 16)
lFileStartTime = readfrombyn(f, "i", 24)
lDataSectionPtr = readfrombyn(f, "i", 40)
lTagSectionPtr = readfrombyn(f, "i", 44)
lNumTagEntries = readfrombyn(f, "i", 48)
lSynchArrayPtr = readfrombyn(f, "i", 92)
lSynchArraySize = readfrombyn(f, "i", 96)
nDataFormat = readfrombyn(f, "h", 100)
nADCNumChannels = readfrombyn(f, "h", 120)
fADCSampleInterval = readfrombyn(f, "f", 122)
fSynchTimeUnit = readfrombyn(f, "f", 130)
lNumSamplesPerEpisode = readfrombyn(f, "i", 138)
lPreTriggerSamples = readfrombyn(f, "i", 142)
lEpisodesPerRun = readfrombyn(f, "i", 146)
fADCRange = readfrombyn(f, "f", 244)
lADCResolution = readfrombyn(f, "i", 252)
nFileStartMillisecs = readfrombyn(f, "h", 366)
nADCPtoLChannelMap = readfrombyn(f, "16h", 378)
nADCSamplingSeq = readfrombyn(f, "16h", 410)
sADCChannelName = readfrombyn(f, "10s"*16, 442)
sADCUnits = readfrombyn(f, "8s"*16, 602)
fADCProgrammableGain = readfrombyn(f, "16f", 730)
fInstrument
= readfrombyn(f, "16f", 922)
fInstrumentOffset = readfrombyn(f, "16f", 986)
fSignalGain = readfrombyn(f, "16f", 1050)
fSignalOffset = readfrombyn(f, "16f", 1114)
nDigitalEnable = readfrombyn(f, "h", 1436)
nActiveDACChannel = readfrombyn(f, "h", 1440)
nDigitalHolding = readfrombyn(f, "h", 1584)
nDigitalInterEpisode = readfrombyn(f, "h", 1586)
nDigitalValue = readfrombyn(f, "10h", 2588)
lDACFilePtr = readfrombyn(f, "2i", 2048)
lDACFileNumEpisodes = readfrombyn(f, "2i", 2056)
fDACCalibrationFactor = readfrombyn(f, "4f", 2074)
fDACCalibrationOffset = readfrombyn(f, "4f", 2090)
nWaveformEnable = readfrombyn(f, "2h", 2296)
nWaveformSource = readfrombyn(f, "2h", 2300)
nInterEpisodeLevel = readfrombyn(f, "2h", 2304)
nEpochType = readfrombyn(f, "20h", 2308)
fEpochInitLevel = readfrombyn(f, "20f", 2348)
fEpochLevelInc = readfrombyn(f, "20f", 2428)
lEpochInitDuration = readfrombyn(f, "20i", 2508)
lEpochDurationInc = readfrombyn(f, "20i", 2588)
nTelegraphEnable = readfrombyn(f, "16h", 4512)
fTelegraphAdditGain = readfrombyn(f, "16f", 4576)
sProtocolPath = readfrombyn(f, "384s", 4898)